In [247]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.cohort.cohort import CohortProcessor

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [248]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'

#
cohort = CohortProcessor(fname_spreadsheet)
cohort.load_database()

#
cohort.ethogram = ethogram

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [249]:
################################################
########### CREATE ROI OF INTEREST #############
################################################
# need to select 1 video and 1 slp for visualization purposes;
# make sure the video is shirnk cropped and matches the traces from the slop also.
cohort.fname_video ='/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/2020_07_26_00_05_52_835502_compressed_defished_shrink_cropped.mp4'
cohort.fname_slp_npy =  '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy'

# cat's workstation; comment out
# cohort.fname_video ='/media/cat/4TBSSD/dan/cohort2/P21_P28/2020_07_26_00_05_52_835502_compressed_defished_shrink_cropped.mp4'
# cohort.fname_slp_npy =  '/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy'


#
cohort.load_video2()

#
cohort.set_roi()

print ("rectangular coords: ", cohort.rect_coords)
print ("circular coords: ", cohort.circle_coords)


ASSUMING VIDEO IS 700 x 900... this only works for cohort2...
(28802, 6, 2)
rectangular coords:  [[280.44292804 500.05687743]
 [576.12531017 636.56270737]]
circular coords:  [[576.12531017 636.56270737]
 [576.12531017 636.56270737]]


In [30]:
################################################
########### COMPUTE CIRCLE OCCUPANCY ###########
################################################

#
behavior_name = 'circular_region'   # "circular_region' or 'rectangular_region"

# select animals
cohort.exclude_huddles = True

# select animals
animal_ids = [0,1,2,3,4,5]
animal_ids = [0]

#
for animal_id in animal_ids:

    #
    #cohort.animals = animal_id
    cohort.animal_ids = animal_id

    #
    cohort.get_circle_occupancy(cohort.animal_ids)

    #
    cohort.format_behavior()
    
    #
    ethogram.generate_ethogram_hourly( 
                                      behavior_name,
                                      cohort,
                                      cohort.exclude_huddles)
print ("DONE")

100%|█████████████████████████████████████████████| 1500/1500 [00:02<00:00, 524.88it/s]


DONE


In [31]:
##########################################
########### SHOW CIRCLE ETHOGRAM #########
##########################################
for animal_id in animal_ids:

    cohort.animal_ids = animal_id
    
    cohort.exclude_huddles = True
    ethogram.plot_ethogram_hourly(
                                 behavior_name,
                                 cohort,
                                 cohort.exclude_huddles,
                                 vmax=50)

In [69]:
print (cohort.rect_coords)

[[775.33126551 277.28694663]
 [899.5        417.58460518]]


In [250]:
################################################
########### COMPUTE RECTANGLE ETHOGRAMS ########
################################################

#
#
cohort.video_frame_rate = 24
cohort.parallel_flag = True

# set the flags for the huddle corrections
cohort.fix_track_flag = True
cohort.interpolate_flag = True
#cohort.load_huddle_tracks()

# # this just loads the _spine.npy files;
cohort.use_nohuddle = True
cohort.load_feature_tracks()

# # this computes mode-based occupancy in the nest with 1min bins
cohort.median_filter_width = cohort.video_frame_rate*5+1 # 1: there is no smotthing; n x frame rate gives you n seconds of somothing (always add 1 to get odd number)
#cohort.n_frames_per_bin = cohort.video_frame_rate  # single second resolution
cohort.n_frames_per_bin = cohort.video_frame_rate #*60  # single second resolution
cohort.n_cores = 24

#
cohort.compute_rectangle_composition()


# of feature tracks:  991 , missing:  0


1001it [03:24,  4.90it/s]                                                              


In [252]:
################################################
########### SHOW RECTANGLE ETHOGRAM ############
################################################

#%matplotlib qt

#
cohort.forward_padding = 20 # padding in percent
cohort.generate_rectangle_composition_ethograms()

#
cohort.show_rectangle_composition_ethogram_all_animals()

#
#cohort.show_huddle_composition_ethogram_full_huddle()

#
#animal_ids = np.arange(6)
# cohort.show_huddle_composition_ethogram_specific_animals(animal_ids)
    

100%|██████████████████████████████████████████████| 991/991 [00:00<00:00, 2992.86it/s]


In [254]:
test = np.load('/home/cat/Downloads/rois/foodhopper.npy')
test.shape

(112, 86400)

In [246]:
##################################################
########### COMPUTE CORRELOGRAMS PER ANIMAL ######
##################################################

print (cohort.rectangle_ethogram.shape)

a1 = 1
a2 = 2

a2_array = [0,1,2,3,4,5]
plt.figure()
clrs = ['black','blue','red','green','magenta','orange']
for a2 in a2_array:

    idx1 = np.arange(a1, 16*7, 7)+1
    idx2 = np.arange(a2, 16*7, 7)+1

    #
    #print (cohort.rectangle_ethogram.shape)
    #print ("idx1: ", idx1.shape)
    eth1 = np.hstack(cohort.rectangle_ethogram[::-1][idx1])
    eth2 = np.hstack(cohort.rectangle_ethogram[::-1][idx2])
    idx3 = np.where(np.isnan(eth1))[0]
    #print ("# of nans: ", idx3.shape)
    #print ("eth1: ", eth1.shape)

    #
    eth1[np.isnan(eth1)]=0
    eth2[np.isnan(eth2)]=0

    n_mins = 1 
    width = n_mins#*60
    text = 'Time (sec)'
    if width > 1:
        idx1 = np.arange(width,eth1.shape[0],width)
        eth1 = np.vstack(np.split(eth1, idx1))
        idx2 = np.arange(width,eth2.shape[0],width)
        eth2 = np.vstack(np.split(eth2, idx2))

        #
        eth1 = np.nansum(eth1,axis=1)
        eth2 = np.nansum(eth2,axis=1)
        text = 'Time (mins)'
        
    # compute the correlograms between each pair of animals
    from scipy.stats import pearsonr

    #
    window = 360
    t = np.arange(-window,window,1)
    from tqdm import trange 

    #
    cc = []
    for z in trange(-window, window,1, desc='computing correlogram'):
        #temp = (np.roll(single_animal_eth,z)*single_animal_eth).sum()
        temp = (np.roll(eth1,z)*eth2).sum()

        cc.append(temp)
    #
    plt.plot(t,cc, label = str(a1)+" vs "+str(a2),
            c=clrs[a2])
plt.plot([0,0],[-1,1],'--',c='black')
plt.legend()
#
plt.title("Animal: "+str(a1))
plt.xlabel(text)
plt.semilogy()
plt.show()




(112, 86400)


computing correlogram: 100%|████████████████████████| 720/720 [00:04<00:00, 147.07it/s]


In [242]:

print (cohort.rectangle_ethogram.shape)

a1 = 0
a2 = 0

#a2_array = [0,1,2,3,4,5]
idx1 = np.arange(a1, 16*7, 7)
idx2 = np.arange(a2, 16*7, 7)

#
plt.figure()
clrs = ['black','blue','red','green','magenta','orange']
cmap = plt.get_cmap("viridis", 16)
ctr=0
for id1, id2 in zip(idx1,idx2):

    #
    print (id1, id2)
    #print (cohort.rectangle_ethogram.shape)
    #print ("idx1: ", idx1.shape)
    eth1 = np.hstack(cohort.rectangle_ethogram[::-1][id1])
    eth2 = np.hstack(cohort.rectangle_ethogram[::-1][id2])
    idx3 = np.where(np.isnan(eth1))[0]
    #print ("# of nans: ", idx3.shape)
    #print ("eth1: ", eth1.shape)

    #
    eth1[np.isnan(eth1)]=0
    eth2[np.isnan(eth2)]=0

    n_mins = 1 
    width = n_mins#*60
    text = 'Time (sec)'
    if width > 1:
        idx1 = np.arange(width,eth1.shape[0],width)
        eth1 = np.vstack(np.split(eth1, idx1))
        idx2 = np.arange(width,eth2.shape[0],width)
        eth2 = np.vstack(np.split(eth2, idx2))

        #
        eth1 = np.nansum(eth1,axis=1)
        eth2 = np.nansum(eth2,axis=1)
        text = 'Time (mins)'
        
    # compute the correlograms between each pair of animals
    from scipy.stats import pearsonr

    #
    window = 360
    t = np.arange(-window,window,1)
    from tqdm import trange 

    #
    cc = []
    for z in trange(-window, window,1, desc='computing correlogram'):
        #temp = (np.roll(single_animal_eth,z)*single_animal_eth).sum()
        temp = (np.roll(eth1,z)*eth2).sum()

        cc.append(temp)
    #
    plt.plot(t,cc, 
             #label = str(a1)+" vs "+str(a2),
            c=cmap(ctr))
    
    ctr+=1
    
    
plt.plot([0,0],[-1,1],'--',c='black')
plt.legend()
#
plt.xlabel(text)
plt.semilogy()
plt.show()



(112, 86400)
0 0


computing correlogram: 100%|███████████████████████| 720/720 [00:00<00:00, 3140.89it/s]


7 7


computing correlogram: 100%|███████████████████████| 720/720 [00:00<00:00, 3121.91it/s]


14 14


...

computing correlogram: 100%|███████████████████████| 720/720 [00:00<00:00, 3147.49it/s]


98 98


computing correlogram: 100%|███████████████████████| 720/720 [00:00<00:00, 3189.65it/s]


105 105


computing correlogram: 100%|███████████████████████| 720/720 [00:00<00:00, 3171.73it/s]

In [240]:
plt.figure()

plt.imshow(cohort.rectangle_ethogram,
          aspect='auto',
          #interpolation='none'
          )

plt.show()

In [19]:
######################################
########### PCA SCATTER PLOTS ########
######################################

#
cohort.animal_ids  = [0,1,2,3,4,5]

# choose one of these behaviors
cohort.behavior_name = 'rectangular_region'
# cohort.behavior_name = 'circular_region'

#
cohort.remove_zeros=True
cohort.show_developmental_trajectories()

#


[0, 1, 2, 3, 4, 5]
(96, 24)
sums:  1704.5969029928478
X_pca:  (96, 3)
removing zeros:  (84,)
